# Data processing for the Organisational Audit Portfolio spreadsheet 2: rename and reduce data

This data reduces redundant parts of the data and then renames what's left for ease of use.

In [68]:
import os
import pandas as pd

### Import data

In [69]:
dir_files = '../data/organisational_audit/processed'
file_excel = 'processed_2019_portfolio_key_indicators_summary.csv'

In [70]:
df = pd.read_csv(
    os.path.join(dir_files, file_excel),
    header=[0, 1, 2, 3, 4, 5],
    index_col=[0, 1, 2, 3]
)

In [71]:
df.columns.names

FrozenList(['SCN', 'Trust name', 'Site name', 'Hospital names', 'hospital_name_2', 'hospital_name_3'])

In [72]:
df.index.names

FrozenList(['Key indicator', 'Response required to meet indicator', 'key_indicator_group', 'question'])

### Header: Check for redundant site names

In [73]:
print(f'Total hospitals: {len(df.columns)}')

for header in df.columns.names:
    n_unique = len(df.columns.get_level_values(header).unique())
    print(f'{header}: {n_unique}')

Total hospitals: 172
SCN: 16
Trust name: 138
Site name: 172
Hospital names: 172
hospital_name_2: 172
hospital_name_3: 172


All hospitals have a unique site name, so we'll move the 'site name' data into another dataframe.

### Header: reduce hospital names

Re-jig "hospital names" column header. In the Excel sheet it is a merged cell and applies to three rows of data. The hospital can have multiple names in English and in Welsh so all should be kept.

Only keep the top row in this DataFrame. Store the other rows' data in a new DataFrame for reference.

Also move the 'site names' data to this new DataFrame.

In [74]:
headers_to_move = ['hospital_name_2', 'hospital_name_3', 'SCN', 'Trust name', 'Site name']

In [75]:
# Combine those rows into one dataframe:
df_hospital_names = pd.DataFrame(
    [df.columns.get_level_values(h) for h in ['Hospital names'] + headers_to_move],
    index=['hospital_name_1', 'hospital_name_2', 'hospital_name_3', 'scn', 'trust', 'site_name']
).T

In [76]:
df_hospital_names.head(3)

,hospital_name_1,hospital_name_2,hospital_name_3,scn,trust,site_name
0,Unnamed: 4_level_3,Unnamed: 4_level_4,Unnamed: 4_level_5,National Results,Unnamed: 4_level_1,Unnamed: 4_level_2
1,Queens Hospital Romford HASU,Queens Hospital Romford SU,Unnamed: 5_level_5,London,"Barking, Havering and Redbridge University Hos...","Barking, Havering and Redbridge University Hos..."
2,Newham General Hospital,Unnamed: 6_level_4,Unnamed: 6_level_5,London,Barts Health NHS Trust,Barts Health NHS Trust (Newham University Hosp...


In [77]:
# Remove the 'National Results' row:
# (assuming it's first in the list)
df_hospital_names = df_hospital_names.iloc[1:]

In [78]:
for col in df_hospital_names.columns:
    mask = df_hospital_names[col].str.startswith('Unnamed')
    df_hospital_names.loc[mask, col] = ''

Save hospital names df to file:

In [79]:
df_hospital_names.to_csv(os.path.join(dir_files, 'hospital_names_trusts.csv'), index=False)

Example of how to access a hospital if you know one of its names but not which column of the DataFrame that name is in:

In [80]:
def find_main_hospital_name(
        df_hospital_names: 'pd.DataFrame',
        name_to_look_up: 'str',
        column_main_name: 'str'='hospital_name_1'
        ):
    # df of True/False for name matches the name_to_look_up:
    df_bool = df_hospital_names.eq(name_to_look_up)
    # series of True/False, one for each row in the dataframe,
    # and the row is True when any value in the row in df_bool is True.
    series_bool = df_bool.any(axis='columns')
    # Use that series as a mask to pick out only the right row,
    # then pick out the value in that row and the right column:
    main_hospital_name = df_hospital_names.loc[series_bool, column_main_name].values[0]
    return main_hospital_name

In [81]:
name_to_look_up = 'Northwick Park Hospital SU'

find_main_hospital_name(df_hospital_names, name_to_look_up)

'Northwick Park Hospital HASU'

Drop the other hospital name headers from the organisational audit dataframe:

In [82]:
df.columns = df.columns.droplevel(headers_to_move)

Rename the 'National results' column:

In [87]:
col = df.columns[df.columns.str.startswith('Unnamed')].values[0]

df = df.rename(columns={col: 'National Results'})

In [88]:
df.head(3)

Hospital names                                                                                                                                                       National results  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                                                      
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN       1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                                            58% (98/169)   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                      1.9 (1.4-2.9)_x000D_\nMedian (IQR)   

Hospital names                                                                                                                     Queens Hospital Romford HASU  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                                
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                                 6   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                               Yes   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                              2.73   

Hospital names                                                                                                                     Newham General Hospital  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                           
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                            6   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                          Yes   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                         1.54   

Hospital names                                                                                                                     Royal London Hospital HASU  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                              
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                               5   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                             Yes   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                            2.85   

Hospital names                                                                                                                     Whipps Cross University Hospital  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                                    
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                                     3   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                                   

## Set up new index names

Make a new set of data that links the new short names with the full names they were once called and any other notes.

Copy the existing index, make a new column for the short name, and save to file.

In the final dataframe, only keep the short names.

In [49]:
print(stop, here, please)

NameError: name 'stop' is not defined

### Split off 'National Results'

This column is formatted too differently from the individual hospitals to keep it in with the rest.

In [37]:
df_national_results = df['National Results']

In [38]:
df_national_results.head(3)

National Results
Key indicator                                      Response required to meet indicator                key_indicator_group question                                                                                             
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                                                 1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                                                                                      58% (98/169)
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                                                                1.9 (1.4-2.9)_x000D_\nMedian (IQR)
                                                   Band 7 nurses WTE per 10 beds                      Staffing/Workforce  NaN                                                                0.5 (0.4-0.8)_x000D_\nMedian (IQR)
2                                                  Key Indicator 2: Presence of a clinical psychol... Staffing/Workforce  NaN                                                                                       7% (12/169)
                                                   Clinical psychologist WTE per 30 beds (qualified)  Staffing/Workforce  NaN                                                                  0.1 (0-0.3)_x000D_\nMedian (IQR)
3                                                  Key Indicator 3:  Out of hours presence of stro... 7-day working       NaN                                                                                     71% (101/142)
                                                   Out of hours                                       7-day working       1.7. Do you have stroke specialist nurses (band...                                      71% (101/142)
                                                   Registered nurses Type 1 Beds (weekdays 10 pm)     7-day working       1.7. Do you have stroke specialist nurses (band...                                                215
                                                   Registered nurses Type 1 Beds (Saturdays)          7-day working       1.7. Do you have stroke specialist nurses (band...                                                243
                                                   Registered nurses Type 1 Beds (Saturdays 10 pm)    7-day working       1.7. Do you have stroke specialist nurses (band...                                                215
                                                   Registered nurses Type 1 Beds (Sundays)            7-day working       1.7. Do you have stroke specialist nurses (band...                                                243
                                                   Registered nurses Type 1 Beds (Sundays 10 pm)      7-day working       1.7. Do you have stroke specialist nurses (band...                                                214
                                                   Registered nurses Type 3 Beds (weekdays 10 pm)     7-day working       1.7. Do you have stroke specialist nurses (band...                                                234
                                                   Registered nurses Type 3 Beds (Saturdays)          7-day working       1.7. Do you have stroke specialist nurses (band...                                                316

Drop the national results from the main organisational audit dataframe:

In [39]:
df = df.drop('National Results', axis='columns)

In [42]:
df.head(3)

Key indicator                                                                              Total Key Indicators Achieved \n(Post 72hrs sites receive points from KIs 3,4, and 6  from their main acute site)  \
Response required to meet indicator                                                                                                                                                                      NaN   
key_indicator_group                                                                                                                                                                                      NaN   
question                                                                                                                                                                                                 NaN   
SCN    Trust name                                         Hospital names                                                                                                                                       
London Barking, Havering and Redbridge University Hosp... Queens Hospital Romford HASU                                                      6                                                                  
       Barts Health NHS Trust                             Newham General Hospital                                                           6                                                                  
                                                          Royal London Hospital HASU                                                        5                                                                  
                                                          Whipps Cross University Hospital                                                  3                                                                  
       Imperial College Healthcare NHS Trust              Charing Cross Hospital HASU                                                       5                                                                  

Key indicator                                                                                                                                                                                                                                                                                            1  \
Response required to meet indicator                                                        Key Indicator 1:  Minimum establishment of band 6 and band 7 nurses per 10 beds (Criterion: Sum of band 6 and 7 (WTE) nurses per 10 stroke unit beds is equal to/above 2.375 per 10 beds for ALL stroke beds.)    
key_indicator_group                                                                                                                                                                                                                                                                     Staffing/Workforce   
question                                                                                                                                                                                                                                                                                               NaN   
SCN    Trust name                                         Hospital names                                                                                                                                                                                                                                     
London Barking, Havering and Redbridge University Hosp... Queens Hospital Romford HASU                                                    Yes                                                                                                                                                                
       Barts Health NHS Trust                             Newham General Hospital                          

## Save results to file